In [1]:
import pandas as pd
import numpy as np


In [4]:
df_train = pd.read_csv("train.tsv",sep="\t")
df_train.head(5)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [5]:
df_test = pd.read_csv("test.tsv",sep="\t")
df_test.head(5)

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [6]:
df_train.groupby("Sentiment").size()

Sentiment
0     7072
1    27273
2    79582
3    32927
4     9206
dtype: int64

In [9]:
X_train = df_train['Phrase']
Y_train = df_train['Sentiment']
X_test = df_test['Phrase']
X_test_phraseid = df_test['PhraseId']
print(X_train.shape,X_test.shape)
X_all = X_train.append(X_test)
print(X_all.shape)

(156060,) (66292,)
(222352,)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_all = vect.fit_transform(X_all)
X_all.shape

(222352, 17729)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_all = tfidf.fit_transform(X_all)
X_all.shape

(222352, 17729)

In [12]:
X_train_all = X_all[:156060]
X_test = X_all[156060:]
print(X_train_all.shape,X_test.shape)

(156060, 17729) (66292, 17729)


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_validation,Y_train,Y_validation = train_test_split(X_train_all,Y_train,test_size=0.2)
print(X_train.shape,X_validation.shape,Y_train.shape,Y_validation.shape)

(124848, 17729) (31212, 17729) (124848,) (31212,)


In [14]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,Y_train)
nb.score(X_validation,Y_validation)

0.57961681404587984

In [16]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(n_estimator=100,learning_rate=0.1)
clf.fit(X_train,Y_train)
clf.score(X_validation,Y_validation)

0.58407022939894915

In [18]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
clf.score(X_validation,Y_validation)

0.62443931821094456

In [20]:
ans = clf.predict(X_test)

In [22]:
res = np.column_stack((X_test_phraseid,ans))
out = pd.DataFrame(data=res,columns=['PhraseId','Sentiment'])
out.to_csv('sol.csv',index=False)